# Identify tags in airline database

## Deep LSTM networks

    - Bidirectional LSTM
    - Stacked LSTM
    - Bidirectional stacked LSTM

In [0]:
# Configure to use tensorboard in colab

#CPU
#!pip install -q tensorflow==2.0.0-alpha0

#GPU
!pip install -q tensorflow-gpu==2.0.0-alpha0

# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook

In [0]:
from __future__ import print_function

import sys
import os 
import numpy as np 
import tensorflow as tf 
print(tf.__version__)

#Show images
import matplotlib.pyplot as plt
%matplotlib inline
# plt configuration
plt.rcParams['figure.figsize'] = (10, 10)


## Dataset transformation


In [0]:
# download data
! wget https://s3-eu-west-1.amazonaws.com/text-mining-course/atis.zip
! unzip atis.zip

# Read data
import pickle
import sys

atis_file = './atis/atis.pkl'

with open(atis_file,'rb') as f:
    if sys.version_info.major==2:
        train, test, dicts = pickle.load(f) #python2.7
    else:
        train, test, dicts = pickle.load(f, encoding='bytes') #python3

#Dictionaries and train test partition
w2idx, ne2idx, labels2idx = dicts[b'words2idx'], dicts[b'tables2idx'], dicts[b'labels2idx']
    
idx2w  = dict((v,k) for k,v in w2idx.items())
idx2la = dict((v,k) for k,v in labels2idx.items())

train_x, _, train_label = train
test_x,  _,  test_label  = test


# Max value of word coding to assign the ID_PAD
ID_PAD = np.max([np.max(tx) for tx in train_x]) + 1
print('ID_PAD: ', ID_PAD)

def context(l, size=3):
    l = list(l)
    lpadded = size // 2 * [ID_PAD] + l + size // 2 * [ID_PAD]
    out = [lpadded[i:(i + size)] for i in range(len(l))]
    return out


# Create train and test X y.
X_trn=[]
for s in train_x:
    X_trn += context(s,size=10)
X_trn = np.array(X_trn)

X_tst=[]
for s in test_x:
    X_tst += context(s,size=10)
X_tst = np.array(X_tst)

print('X trn shape: ', X_trn.shape)
print('X_tst shape: ',X_tst.shape)


y_trn=[]
for s in train_label:
    y_trn += list(s)
y_trn = np.array(y_trn)
print('y_trn shape: ',y_trn.shape)

y_tst=[]
for s in test_label:
    y_tst += list(s)
y_tst = np.array(y_tst)
print('y_tst shape: ',y_tst.shape)


print('Num labels: ',len(set(y_trn)))
print('Num words: ',len(set(idx2w)))

In [0]:
# data attributes
input_seq_length = X_trn.shape[1]
input_vocabulary_size = len(set(idx2w)) + 1
output_length = 127

#Model parameters
embedding_size=64
num_hidden_lstm = 128


# Bidirectional LSTM model


In [0]:
# build the model: Simple LSTM with embedings
print('Build model 1')
seq_input = tf.keras.layers.Input(shape=([input_seq_length]), name='prev') 
    
embeds = tf.keras.layers.Embedding(input_vocabulary_size, embedding_size)(seq_input)

# Bidirectional LSTM
forwards  = tf.keras.layers.LSTM(num_hidden_lstm, return_sequences=False, name='Forward')(embeds)
backwards = tf.keras.layers.LSTM(num_hidden_lstm, return_sequences=False, go_backwards=True, name='Backward')(embeds)
rnn_out = tf.keras.layers.concatenate([forwards, backwards], axis=-1, name='Forward_Backward')

output = tf.keras.layers.Dense(output_length, activation='softmax')(rnn_out)

model1 = tf.keras.models.Model(inputs=seq_input, outputs=output)
model1.summary()

# Optimizer
adam_optimizer = tf.keras.optimizers.Adam()
model1.compile(loss='sparse_categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

#Fit model
tb_callback_bilstm = tf.keras.callbacks.TensorBoard(log_dir='./tensorboard/airline/BILSTM/')
history1 = model1.fit(X_trn, y_trn, batch_size=128, epochs=20,
                      validation_data=(X_tst, y_tst), callbacks=[tb_callback_bilstm])


# Stacked LSTM model

In [0]:
# build the model: Simple LSTM with embedings
print('Build model 1')
seq_input = tf.keras.layers.Input(shape=([input_seq_length]), name='prev') 
    
embeds = tf.keras.layers.Embedding(input_vocabulary_size, embedding_size)(seq_input)

# Stacked LSTM
forwards1 = tf.keras.layers.LSTM(num_hidden_lstm, return_sequences=True,  name='Forward1')(embeds)
forwards2 = tf.keras.layers.LSTM(num_hidden_lstm, return_sequences=False, name='Forward2')(forwards1)

output = tf.keras.layers.Dense(output_length, activation='softmax')(forwards2)

model2 = tf.keras.models.Model(inputs=seq_input, outputs=output)
model2.summary()

# Optimizer
adam_optimizer = tf.keras.optimizers.Adam()
model2.compile(loss='sparse_categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])


#Fit model
tb_callback_lstm2 = tf.keras.callbacks.TensorBoard(log_dir='./tensorboard/airline/LSTM2/')
history2 = model2.fit(X_trn, y_trn, batch_size=128, epochs=20,
                      validation_data=(X_tst, y_tst), callbacks=[tb_callback_lstm2])


# Bidirectional stacked LSTM model

In [0]:
# build the model: Bidirectional stacked LSTM with embedings
print('Build model 1')
seq_input = tf.keras.layers.Input(shape=([input_seq_length]), name='prev') 
    
embeds = tf.keras.layers.Embedding(input_vocabulary_size, embedding_size)(seq_input)

# Bidirectional LSTM 1
forwards1  = tf.keras.layers.LSTM(num_hidden_lstm, return_sequences=True, name='Forward1')(embeds)
backwards1 = tf.keras.layers.LSTM(num_hidden_lstm, return_sequences=True, go_backwards=True, name='Backward1')(embeds)
rnn_out1 = tf.keras.layers.concatenate([forwards1, backwards1], axis=-1, name='Forward_Backward1')

# Bidirectional LSTM 1
forwards2  = tf.keras.layers.LSTM(num_hidden_lstm, return_sequences=False, name='Forward2')(rnn_out1)
backwards2 = tf.keras.layers.LSTM(num_hidden_lstm, return_sequences=False, go_backwards=True, name='Backward2')(rnn_out1)
rnn_out2 = tf.keras.layers.concatenate([forwards2, backwards2], axis=-1, name='Forward_Backward2')


output = tf.keras.layers.Dense(output_length, activation='softmax')(rnn_out2)

model3 = tf.keras.models.Model(inputs=seq_input, outputs=output)
model3.summary()


In [0]:
# Optimizer
adam_optimizer = tf.keras.optimizers.Adam()
model3.compile(loss='sparse_categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])


#Fit model
tb_callback_bilstm2 = tf.keras.callbacks.TensorBoard(log_dir='./tensorboard/airline/BILSTM2/')
history3 = model3.fit(X_trn, y_trn, batch_size=128, epochs=20,
                      validation_data=(X_tst, y_tst), callbacks=[tb_callback_bilstm2])


In [0]:
# Compare validation of the 3 models
plt.rcParams['figure.figsize'] = (10, 10)

plt.plot(history1.history['val_accuracy'], label='BLSTM')
plt.plot(history2.history['val_accuracy'], label='Stacked LSTM')
plt.plot(history3.history['val_accuracy'], label='Stacked BLSTM')
plt.legend(loc='lower right')
plt.show()

In [0]:
# Start tensorboard
%tensorboard --logdir ./tensorboard/airline

## Retrieve the learned embeddings to show into the tensorflow projector

In [0]:
# List layers to identigy the embedding layer
model2.layers

In [0]:
# Get the trined weights of the embedding layer
model_layers = model2.layers[1]
weights = model_layers.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

In [0]:
idx2w

In [0]:
# Generate files to load into the projector

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in idx2w.keys():
  word = idx2w[word_num]
  embeddings = weights[word_num]
  out_m.write(str(word) + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()



In [0]:
# download to local

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')



## Load the files into the projector

http://projector.tensorflow.org/

Execute t-sne and examine results.